## Read NZX50 Stocks File and Import Packages

In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
from package import my_functions
warnings.filterwarnings('ignore')

url = 'https://www.interest.co.nz/nzx50'
nzx50 = pd.read_html(url,  flavor='html5lib')[0]

## Grab symbols from dataframe and convert into a list for YFinance

In [2]:
symbols_list = nzx50["Profile"].unique().tolist()
del symbols_list[-1]
symbols_list_nz = [symbol + ".NZ" for symbol in symbols_list]
symbols_list_nz = [s.replace('FHP.NZ', 'FPH.NZ') for s in symbols_list_nz] # had to change ticker to match yfinance ticker for FHP

In [3]:
end_date = '2024-03-10'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*2) # use last 2 years of data

df = yf.download(tickers=symbols_list_nz,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker'] # convert into multiindex df

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  50 of 50 completed


Price              adj close      close       high        low       open  \
date       ticker                                                          
2022-03-11 AIA.NZ   7.044579   7.140000   7.200000   7.090000   7.120000   
           AIR.NZ   0.774377   0.863834   0.885276   0.845454   0.876086   
           ANZ.NZ  24.075434  27.381542  27.609722  27.123602  27.480753   
           ARG.NZ   1.225725   1.390000   1.405000   1.375000   1.390000   
           ARV.NZ   1.433682   1.540000   1.570000   1.530000   1.570000   
...                      ...        ...        ...        ...        ...   
2024-03-08 VGL.NZ   1.680000   1.680000   1.680000   1.620000   1.620000   
           VHP.NZ   2.130000   2.130000   2.140000   2.115000   2.140000   
           VSL.NZ   8.270000   8.270000   8.440000   8.250000   8.250000   
           WBC.NZ  29.600000  29.600000  29.610001  28.700001  28.700001   
           WHS.NZ   1.258064   1.300000   1.320000   1.300000   1.320000   

Price               volume  
date       ticker           
2022-03-11 AIA.NZ  1025995  
           AIR.NZ  3634730  
           ANZ.NZ    25139  
           ARG.NZ   252586  
           ARV.NZ   267235  
...                    ...  
2024-03-08 VGL.NZ   137018  
           VHP.NZ   129645  
           VSL.NZ     7107  
           WBC.NZ    95334  
           WHS.NZ    29746  

[24900 rows x 6 columns]

In [4]:
index_names = df.index.names
column_names = df.columns.names

print("Index Names:", index_names)
print("Column Names:", column_names)

Index Names: ['date', 'ticker']
Column Names: ['Price']


In [5]:
columns = df.columns
print(columns)

Index(['adj close', 'close', 'high', 'low', 'open', 'volume'], dtype='object', name='Price')


## 2. Calculate Features and Technical indicators for each stock
* Garman - Klass Volatility is a volatility estimator that incorporates low, high, and close prices of a security
* RSI - Relative Strength Index measures that speed and magnitude of a security's recent price change
* Bollinger Bands - Helps determine whether prices are high or low on a relative basis
* ATR - Average True Range, price volatility indicator showing the average price variation of assets within a given time period
* MACD - Moving average convergence/divergence is a techical indicator to identify market entry points for buying/selling
* Dollar volume - number of shares traded times the price of the share

$$
    \text{Garman-Klass Volatilty} = \frac{(\text{ln}(\text{High})-\text{ln}(\text{Low}))^2}{2} - (2\text{ln}(2) -1)(\text{ln}(\text{Adj Close}) - \text{ln}(\text{Open}))^2
$$

In [6]:
my_functions.garman_klass_vol(df, "high", "low", "adj close", "open")
my_functions.rsi(df, "adj close")
my_functions.bollinger_bands(df, "adj close")
my_functions.compute_atr(df, 'high', 'low', 'close')
df['atr1'] = df.groupby(level=1, group_keys=False).apply(
    lambda group: my_functions.compute_atr(group, 'high', 'low', 'close')
)
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(my_functions.compute_macd)
my_functions.dollar_volume(df, 'adj close', 'volume')

In [7]:
df

Price              adj close      close       high        low       open  \
date       ticker                                                          
2022-03-11 AIA.NZ   7.044579   7.140000   7.200000   7.090000   7.120000   
           AIR.NZ   0.774377   0.863834   0.885276   0.845454   0.876086   
           ANZ.NZ  24.075434  27.381542  27.609722  27.123602  27.480753   
           ARG.NZ   1.225725   1.390000   1.405000   1.375000   1.390000   
           ARV.NZ   1.433682   1.540000   1.570000   1.530000   1.570000   
...                      ...        ...        ...        ...        ...   
2024-03-08 VGL.NZ   1.680000   1.680000   1.680000   1.620000   1.620000   
           VHP.NZ   2.130000   2.130000   2.140000   2.115000   2.140000   
           VSL.NZ   8.270000   8.270000   8.440000   8.250000   8.250000   
           WBC.NZ  29.600000  29.600000  29.610001  28.700001  28.700001   
           WHS.NZ   1.258064   1.300000   1.320000   1.300000   1.320000   

Price               volume  garman_klass_vol        rsi    bb_low    bb_mid  \
date       ticker                                                             
2022-03-11 AIA.NZ  1025995          0.000075        NaN       NaN       NaN   
           AIR.NZ  3634730         -0.004824        NaN       NaN       NaN   
           ANZ.NZ    25139         -0.006603        NaN       NaN       NaN   
           ARG.NZ   252586         -0.005878        NaN       NaN       NaN   
           ARV.NZ   267235         -0.002854        NaN       NaN       NaN   
...                    ...               ...        ...       ...       ...   
2024-03-08 VGL.NZ   137018          0.000150  57.467567  0.929810  0.959052   
           VHP.NZ   129645          0.000061  51.622335  1.105560  1.133823   
           VSL.NZ     7107          0.000257  58.741618  2.132727  2.185009   
           WBC.NZ    95334          0.000119  80.937582  3.269012  3.346795   
           WHS.NZ    29746         -0.000776  36.544815  0.796603  0.830936   

Price               bb_high      atr1      macd  dollar_volume  
date       ticker                                               
2022-03-11 AIA.NZ       NaN       NaN       NaN       7.227702  
           AIR.NZ       NaN       NaN       NaN       2.814650  
           ANZ.NZ       NaN       NaN       NaN       0.605232  
           ARG.NZ       NaN       NaN       NaN       0.309601  
           ARV.NZ       NaN       NaN       NaN       0.383130  
...                     ...       ...       ...            ...  
2024-03-08 VGL.NZ  0.988294  0.087438  0.423001       0.230190  
           VHP.NZ  1.162086 -0.425570 -0.138494       0.276144  
           VSL.NZ  2.237292 -0.276156  0.855618       0.058775  
           WBC.NZ  3.424578  1.488052  2.600949       2.821886  
           WHS.NZ  0.865269 -0.297073 -0.725573       0.037422  

[24900 rows x 14 columns]

## 3. Agregate to monthly level on technical indicators 
* To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency

In [15]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]


data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                                                          df.unstack()[last_cols].resample('M').last().stack('ticker')], axis=1).dropna() 

data

dollar_volume  adj close  garman_klass_vol        rsi  \
date       ticker                                                          
2022-04-30 AIA.NZ       9.525962   7.730290          0.000184  58.188537   
           AIR.NZ       4.291557   0.793351         -0.004005  53.816038   
           ANZ.NZ       0.506043  26.038101         -0.005742  54.349621   
           ARG.NZ       0.510359   1.164691         -0.005703  37.897964   
           ARV.NZ       0.787794   1.526778         -0.000787  49.393850   
...                          ...        ...               ...        ...   
2024-03-31 VGL.NZ       0.794222   1.680000          0.000150  57.467567   
           VHP.NZ       0.491593   2.130000          0.000061  51.622335   
           VSL.NZ       0.056488   8.270000          0.000257  58.741618   
           WBC.NZ       1.255420  29.600000          0.000119  80.937582   
           WHS.NZ       0.191757   1.258064         -0.000776  36.544815   

                     bb_low    bb_mid   bb_high      atr1      macd  
date       ticker                                                    
2022-04-30 AIA.NZ  2.133144  2.155457  2.177769  0.760492  0.631985  
           AIR.NZ  0.525439  0.568056  0.610674  2.587225  0.588923  
           ANZ.NZ  3.274660  3.300577  3.326494 -0.265739  0.275523  
           ARG.NZ  0.770475  0.793023  0.815571  0.963290 -0.977686  
           ARV.NZ  0.915821  0.945272  0.974724  0.578546  0.488126  
...                     ...       ...       ...       ...       ...  
2024-03-31 VGL.NZ  0.929810  0.959052  0.988294  0.087438  0.423001  
           VHP.NZ  1.105560  1.133823  1.162086 -0.425570 -0.138494  
           VSL.NZ  2.132727  2.185009  2.237292 -0.276156  0.855618  
           WBC.NZ  3.269012  3.346795  3.424578  1.488052  2.600949  
           WHS.NZ  0.796603  0.830936  0.865269 -0.297073 -0.725573  

[1200 rows x 9 columns]

* Calculate 2-year rolling average of dollar volume for each stocks before filtering

In [10]:
# This calculates the moving average over a window of 3 months
data['dollar_volume'] = (data['dollar_volume'].unstack('ticker').rolling(3).mean().stack())

In [16]:
# ranks stock by dollar volume
data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data

dollar_volume  adj close  garman_klass_vol        rsi  \
date       ticker                                                          
2022-04-30 AIA.NZ       9.525962   7.730290          0.000184  58.188537   
           AIR.NZ       4.291557   0.793351         -0.004005  53.816038   
           ANZ.NZ       0.506043  26.038101         -0.005742  54.349621   
           ARG.NZ       0.510359   1.164691         -0.005703  37.897964   
           ARV.NZ       0.787794   1.526778         -0.000787  49.393850   
...                          ...        ...               ...        ...   
2024-03-31 VGL.NZ       0.794222   1.680000          0.000150  57.467567   
           VHP.NZ       0.491593   2.130000          0.000061  51.622335   
           VSL.NZ       0.056488   8.270000          0.000257  58.741618   
           WBC.NZ       1.255420  29.600000          0.000119  80.937582   
           WHS.NZ       0.191757   1.258064         -0.000776  36.544815   

                     bb_low    bb_mid   bb_high      atr1      macd  \
date       ticker                                                     
2022-04-30 AIA.NZ  2.133144  2.155457  2.177769  0.760492  0.631985   
           AIR.NZ  0.525439  0.568056  0.610674  2.587225  0.588923   
           ANZ.NZ  3.274660  3.300577  3.326494 -0.265739  0.275523   
           ARG.NZ  0.770475  0.793023  0.815571  0.963290 -0.977686   
           ARV.NZ  0.915821  0.945272  0.974724  0.578546  0.488126   
...                     ...       ...       ...       ...       ...   
2024-03-31 VGL.NZ  0.929810  0.959052  0.988294  0.087438  0.423001   
           VHP.NZ  1.105560  1.133823  1.162086 -0.425570 -0.138494   
           VSL.NZ  2.132727  2.185009  2.237292 -0.276156  0.855618   
           WBC.NZ  3.269012  3.346795  3.424578  1.488052  2.600949   
           WHS.NZ  0.796603  0.830936  0.865269 -0.297073 -0.725573   

                   dollar_vol_rank  
date       ticker                   
2022-04-30 AIA.NZ              5.0  
           AIR.NZ             10.0  
           ANZ.NZ             35.0  
           ARG.NZ             34.0  
           ARV.NZ             26.0  
...                            ...  
2024-03-31 VGL.NZ             26.0  
           VHP.NZ             34.0  
           VSL.NZ             50.0  
           WBC.NZ             19.0  
           WHS.NZ             47.0  

[1200 rows x 10 columns]

In [17]:
data = data.drop(['dollar_volume', 'dollar_vol_rank'], axis=1)
data

adj close  garman_klass_vol        rsi    bb_low    bb_mid  \
date       ticker                                                               
2022-04-30 AIA.NZ   7.730290          0.000184  58.188537  2.133144  2.155457   
           AIR.NZ   0.793351         -0.004005  53.816038  0.525439  0.568056   
           ANZ.NZ  26.038101         -0.005742  54.349621  3.274660  3.300577   
           ARG.NZ   1.164691         -0.005703  37.897964  0.770475  0.793023   
           ARV.NZ   1.526778         -0.000787  49.393850  0.915821  0.945272   
...                      ...               ...        ...       ...       ...   
2024-03-31 VGL.NZ   1.680000          0.000150  57.467567  0.929810  0.959052   
           VHP.NZ   2.130000          0.000061  51.622335  1.105560  1.133823   
           VSL.NZ   8.270000          0.000257  58.741618  2.132727  2.185009   
           WBC.NZ  29.600000          0.000119  80.937582  3.269012  3.346795   
           WHS.NZ   1.258064         -0.000776  36.544815  0.796603  0.830936   

                    bb_high      atr1      macd  
date       ticker                                
2022-04-30 AIA.NZ  2.177769  0.760492  0.631985  
           AIR.NZ  0.610674  2.587225  0.588923  
           ANZ.NZ  3.326494 -0.265739  0.275523  
           ARG.NZ  0.815571  0.963290 -0.977686  
           ARV.NZ  0.974724  0.578546  0.488126  
...                     ...       ...       ...  
2024-03-31 VGL.NZ  0.988294  0.087438  0.423001  
           VHP.NZ  1.162086 -0.425570 -0.138494  
           VSL.NZ  2.237292 -0.276156  0.855618  
           WBC.NZ  3.424578  1.488052  2.600949  
           WHS.NZ  0.865269 -0.297073 -0.725573  

[1200 rows x 8 columns]

## 4. Calculate Monthly Returns for different time horizons as features
* To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is returns over various monthly periods as identified by lags

In [18]:
def calculate_returns(df):
    
    outlier_cutoff = 0.005
    
    lags = [1, 2, 3, 6, 9, 12]
    
    for lag in lags:
    
        df[f'return_{lag}m'] = (df['adj close']
                               .pct_change(lag)
                               .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                               .add(1)
                               .pow(1/lag)
                               .sub(1))
    return df

data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

adj close  garman_klass_vol        rsi    bb_low    bb_mid  \
date       ticker                                                               
2023-04-30 AIA.NZ   8.721860          0.000096  57.278008  2.241463  2.259921   
           AIR.NZ   0.681296         -0.006078  43.363056  0.518453  0.525539   
           ANZ.NZ  24.416733         -0.001334  54.174699  3.156557  3.211139   
           ARG.NZ   1.055104         -0.001657  49.025284  0.714796  0.722836   
           ARV.NZ   1.006719          0.000218  52.898565  0.637037  0.679251   
...                      ...               ...        ...       ...       ...   
2024-03-31 VGL.NZ   1.680000          0.000150  57.467567  0.929810  0.959052   
           VHP.NZ   2.130000          0.000061  51.622335  1.105560  1.133823   
           VSL.NZ   8.270000          0.000257  58.741618  2.132727  2.185009   
           WBC.NZ  29.600000          0.000119  80.937582  3.269012  3.346795   
           WHS.NZ   1.258064         -0.000776  36.544815  0.796603  0.830936   

                    bb_high      atr1      macd  return_1m  return_2m  \
date       ticker                                                       
2023-04-30 AIA.NZ  2.278380  0.535657  0.080221   0.016092   0.005705   
           AIR.NZ  0.532624 -0.465916 -0.095385  -0.006536  -0.019171   
           ANZ.NZ  3.265721 -0.395481  0.559996   0.065367  -0.014357   
           ARG.NZ  0.730877 -0.658693  0.142219   0.009009   0.009532   
           ARV.NZ  0.721466 -0.110009  0.744462   0.118279   0.014743   
...                     ...       ...       ...        ...        ...   
2024-03-31 VGL.NZ  0.988294  0.087438  0.423001   0.043478   0.015222   
           VHP.NZ  1.162086 -0.425570 -0.138494   0.016867  -0.014575   
           VSL.NZ  2.237292 -0.276156  0.855618  -0.003614   0.022412   
           WBC.NZ  3.424578  1.488052  2.600949   0.059034   0.070078   
           WHS.NZ  0.865269 -0.297073 -0.725573  -0.022556  -0.095782   

                   return_3m  return_6m  return_9m  return_12m  
date       ticker                                               
2023-04-30 AIA.NZ   0.013159   0.023278   0.019038    0.010108  
           AIR.NZ  -0.006494  -0.004320   0.024730   -0.012609  
           ANZ.NZ  -0.013114  -0.007003   0.006049   -0.005205  
           ARG.NZ  -0.003970  -0.004669  -0.012374   -0.008128  
           ARV.NZ  -0.035745  -0.028209  -0.037719   -0.033272  
...                      ...        ...        ...         ...  
2024-03-31 VGL.NZ   0.006024   0.026025  -0.003253    0.015590  
           VHP.NZ  -0.008249   0.004793  -0.006518   -0.004020  
           VSL.NZ   0.002183   0.006376  -0.002815    0.005379  
           WBC.NZ   0.061179   0.048405   0.030403    0.024715  
           WHS.NZ  -0.068808  -0.039258  -0.028204   -0.025808  

[600 rows x 14 columns]

## 5. Download Fama-French Factors and Calculate Rolling Factor Betas
* Utilize Fama--French data to estimate the exposure of assets to common risk factors using linear regression
* The five Fama--French Factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Therefore, it is beneficial to include past factor exposures as financial features in models.
* Access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression